## ANALIZA PODATKOV

### 1.NALOGA

Iz podatkov v tabeli filmi izračunaj naslednje:

(a) Kateri film (oz. filmi, če jih je več) je prejel največ nominacij ter koliko. Odgovor shrani v
spremenljivko odgovor1a.

In [2]:
import pandas as pd 

podatki = pd.read_csv('filmi.csv')
najvec_nominacij = podatki['stevilo'].max()
najvec = podatki[podatki['stevilo'] == najvec_nominacij]
odgovor1a = najvec[['naslov', 'stevilo']].drop_duplicates()
odgovor1a = odgovor1a.reset_index(drop=True)
print(odgovor1a)


                                          naslov  stevilo
0                                         Becket       11
1                                        Ben-Hur       11
2                                 Johnny Belinda       11
3  The Lord of the Rings: The Return of the King       11
4                           The Power of the Dog       11
5                                        Titanic       11


(b) Število filmov, ki so bili nominirani vsaj trikrat in so osvojili oskarja za vsaj polovico nominacij. Odgovor shrani v spremenljivko odgovor1b.

In [12]:
zdruzeni_po_filmih = podatki.groupby('naslov')
ustrezni_filmi = zdruzeni_po_filmih.filter(
    lambda group: (group['stevilo'].sum() >= 3 and 
                   group.loc[group['oskar'] == True, 'stevilo'].sum() >= 0.5 * group['stevilo'].sum())
)
odgovor1b = len(ustrezni_filmi['naslov'].unique())
print(odgovor1b)


150


(c) Desetletje in stoletje, v katerem je bilo nagrajenih največ različnih filmov ter koliko. Odgovor izpiši z nizom. Opomba: npr. obdobje 1970- 1979 je 8. desetletje 20. stoletja.

In [ ]:
# Dodaj stolpca za desetletje in stoletje
podatki['desetletje'] = (podatki['leto'] // 10) * 10
podatki['stoletje'] = ((podatki['leto'] - 1) // 100) + 1

# Filtriraj nagrajene filme (oskar == True)
nagrajeni_filmi = podatki[podatki['oskar'] == True]

# Združi po desetletju in stoletju ter preštej unikatne naslove
skupaj = nagrajeni_filmi.groupby(['desetletje', 'stoletje'])['naslov'].nunique()

# Najdi desetletje in stoletje z največ nagrajenimi filmi
najvec_ind = skupaj.idxmax()  # Tuple (desetletje, stoletje)
najvec_st = skupaj.max()      # Največje število filmov

# Oblikuj rezultat
desetletje = najvec_ind[0]
stoletje = najvec_ind[1]
odgovor1c = (
    f"V letih {desetletje}-{desetletje + 9}, kar je {desetletje // 10}. desetletje "
    f"in {stoletje}. stoletje, je bilo nagrajenih {najvec_st} filmov."
)

# Izpiši rezultat
print(odgovor1c)


V letih 1940-1949, kar je 194. desetletje in 20. stoletje, je bilo nagrajenih 174 filmov.


(d) Nekateri naslovi filmov se ponavljajo. Kateri izmed filmov, katerih naslovi se ponovijo na
tanko dvakrat, imajo tudi enako število prejetih oskarjev? Seznam filmov shrani v spremenljivko
odgovor1d.

In [3]:
import pandas as pd

# Predpostavimo, da so podatki že naloženi v DataFrame z imenom 'podatki'
# Stolpci: 'naslov', 'leto', 'oskar', 'stevilo'

# 1. Filtriramo vrstice, kjer je 'oskar' True
podatki_oskar = podatki[podatki['oskar'] == True]

# 2. Združimo podatke po naslovih in preštejemo število pojavitev ter vsoto oskarjev
df_skupaj = (
    podatki_oskar.groupby('naslov')
    .agg(stevilo_pojavitev=('naslov', 'size'), vsota_oskarjev=('stevilo', 'sum'))
    .reset_index()
)

# 3. Filtriramo naslove, ki se pojavijo natanko dvakrat
df_dvakrat = df_skupaj[df_skupaj['stevilo_pojavitev'] == 2]

# 4. Preverimo, kateri izmed teh imajo enako število prejetih oskarjev
odgovor1d = df_dvakrat[df_dvakrat['vsota_oskarjev'].duplicated(keep=False)]['naslov'].tolist()

# Rezultat
print(odgovor1d)


['Cyrano de Bergerac', 'King Kong', 'The Fly', 'The Old Man and the Sea', 'Up']


### 2.NALOGA

Iz podatkov v tabelah tuji filmi, filmi in populacija izraČunaj naslednje:

(a) Število vseh nominacij in število vseh dobljenih oskarjev (tudi oskar za najboljši mednarodni celovečerni film) glede na leto za države iz tabele tuji filmi z izjemo ZDA. Rezultat predstavi s tabelo, ki ima štiri stolpce: drzava, leto, st nominacij in st oskarjev. T  abelo shrani v datoteko tabela2a.csv.
 
Namig: v tabeli filmi so upoštevani vsi oskarji, v tabeli tuji pa vsi, razen oskarja za najboljši mednarodni celovečerni film. 

In [26]:
import pandas as pd

def nominacije_in_nagrade(tuji_filmi_d, filmi_d, koncna_d):
    # Naloži podatke iz CSV datotek
    pod_tuji = pd.read_csv(tuji_filmi_d)
    pod_filmi = pd.read_csv(filmi_d)

    # Filtriraj tuje filme, izključujoč ZDA
    filtrirano = pod_tuji[pod_tuji['drzava'] != 'USA']

    # Združi podatke o nominacijah in oskarjih
    skupaj = filtrirano.merge(
        pod_filmi,
        how='left',
        left_on=['naslov_nom', 'leto'],
        right_on=['naslov', 'leto']
    )

    # Združevanje po državi in letu ter izračun števila nominacij in oskarjev
    rez = skupaj.groupby(['drzava', 'leto']).agg(
        st_nominacij=pd.NamedAgg(column='stevilo', aggfunc='sum'),
        st_oskarjev=pd.NamedAgg(column='oskar', aggfunc='sum')
    ).reset_index()

    # Shranjevanje rezultata v CSV datoteko
    rez.to_csv(koncna_d, index=False)
    
    return rez, skupaj

# Definiraj poti do datotek
tuji_filmi_dat = 'tuji_filmi.csv'
filmi_dat = 'filmi.csv'
populacija_dat = 'populacija.csv'
koncna = 'tabela2a.csv'

# Klic funkcije za izračun in shranjevanje
nominacije_in_nagrade(tuji_filmi_dat, filmi_dat, koncna)

print(f"Rezultati so shranjeni v datoteko {koncna}.")


Rezultati so shranjeni v datoteko tabela2a.csv.


(b) Največ koliko nominacij je prejela država na eni podelitvi? Rezultat shrani v spremenljivko odgovor2b.

In [19]:
rezultatb, zdruzeni_podatki = nominacije_in_nagrade(tuji_filmi_dat, filmi_dat, koncna)
odgovor2b = rezultatb['st_nominacij'].max()
print(odgovor2b)

54.0


(c) Največ koliko oskarjev je prejela država na eni podelitvi? Rezultat shrani v spremenljivko odgovor2c.

In [20]:
rezultatc, zdruzeni_podatki  = nominacije_in_nagrade(tuji_filmi_dat, filmi_dat, koncna)
odgovor2c = rezultatc['st_oskarjev'].max()
print(odgovor2c)

6


(d) Kateri film je prejel oskarje iz prejšnje točke? Rezultat shrani v spremenljivko odgovor2d.

In [22]:
rezultatd, zdruzeni_podatki = nominacije_in_nagrade(tuji_filmi_dat, filmi_dat, koncna)

film_max_oskarjev = zdruzeni_podatki[zdruzeni_podatki['oskar'] == True]
film_max_oskarjev = film_max_oskarjev.loc[film_max_oskarjev['stevilo'].idxmax(), 'naslov_nom']
odgovor2d = film_max_oskarjev
print(odgovor2d)

The Artist


(e) Za države iz tabele tuji filmi z izjemo ZDA izračunaj delež uspešnih nominacij (nominacija je uspešna, če film prejme oskarja). Rezultat predstavi s tabelo, ki ima dva stolpca: drzava in delez. Tabelo shrani v datoteko tabela2e.csv. Katera država ima najmanjši delež uspešnih nominacij? Rezultat shrani v spremenljivko odgovor2e.

In [25]:
rezultate, zdruzeni_podatki = nominacije_in_nagrade(tuji_filmi_dat, filmi_dat, koncna)

def delez_uspesnih_nominacij(skupaj, koncnae):
    # Izračun deleža uspešnih nominacij po državah
    delez = skupaj.groupby('drzava').apply(
        lambda x: x['oskar'].sum() / x['stevilo'].sum() if x['stevilo'].sum() > 0 else 0
    ).reset_index(name='delez')

    # Shranjevanje rezultata v CSV datoteko
    delez.to_csv(koncnae, index=False)

    # Država z najmanjšim deležem uspešnih nominacij
    najmanjsi_delez_drzava = delez.loc[delez['delez'].idxmin()]
    return delez, najmanjsi_delez_drzava

koncna2e = 'tabela2e.csv'
delez_tabela, odgovor2e = delez_uspesnih_nominacij(zdruzeni_podatki, koncna2e)
print(odgovor2e)

drzava    Czech Republic
delez                0.0
Name: 5, dtype: object


C:\Users\ulabu\AppData\Local\Temp\ipykernel_19128\1603551888.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  delez = skupaj.groupby('drzava').apply(


(f) Število nominacij na milijon prebivalcev. Rezultat predstavi s tabelo, ki ima dva stolpca: drzava in nominacije_na_mio. Uporabi podatke o populaciji iz leta 2015,če podatka ni,naj države ne bo v končni tabeli.Tabelo shrani v datoteko tabela2f.csv

In [28]:
def nominacije_na_milijon(skupaj, populacija_d, koncnaf):
    # Naloži podatke o populaciji
    populacija = pd.read_csv(populacija_d)

    # Filtriraj populacijo na leto 2015
    populacija_2015 = populacija[(populacija['leto'] == 2015)]

    # Združi podatke o populaciji in nominacijah
    nominacije_populacija = skupaj.groupby('drzava').agg(
        st_nominacij=pd.NamedAgg(column='stevilo', aggfunc='sum')
    ).reset_index()

    nominacije_populacija = nominacije_populacija.merge(
        populacija_2015[['enota', 'vrednost']],
        how='inner',
        left_on='drzava',
        right_on='enota'
    )

    # Izračun nominacij na milijon prebivalcev
    nominacije_populacija['nominacije_na_mio'] = nominacije_populacija['st_nominacij'] / (nominacije_populacija['vrednost'] / 1_000_000)

    # Shranjevanje rezultata v CSV datoteko
    rezultat = nominacije_populacija[['drzava', 'nominacije_na_mio']]
    rezultat.to_csv(koncnaf, index=False)

    return rezultat

# Izračun nominacij na milijon prebivalcev
koncna_2f = 'tabela2f.csv'
odgovor2f = nominacije_na_milijon(zdruzeni_podatki, populacija_dat, koncna_2f)
print(odgovor2f)

            drzava  nominacije_na_mio
0          Algeria       7.062147e+04
1          Algeria       1.929012e+03
2          Algeria       7.692308e+05
3        Argentina       4.371585e+04
4        Argentina       2.719978e+02
5        Argentina       1.179941e+05
6           Brazil       4.662005e+04
7           Brazil       9.596929e+02
8           Brazil       2.000000e+06
9            Chile       4.576659e+04
10           Chile       6.132148e+02
11           Chile       1.089918e+05
12           China       5.405405e+05
13           China       1.628576e+03
14           China       2.307692e+07
15          France       9.284818e+05
16          France       6.893982e+03
17          France       4.431138e+06
18         Germany       7.642487e+05
19         Germany       1.678998e+04
20         Germany       1.372093e+07
21          Greece       6.410256e+04
22          Greece       1.581778e+03
23          Greece       1.773050e+05
24           India       6.097561e+04
25          

(g) Kateri film iz tabele tuji_filmi so prejeli tudi oskarja za najboljši mednarodni celovečerni film? Seznam filmov shrani v spremenljivko odgovor2g.

In [44]:
def oskarji_za_najboljsi_film(tuji_filmi, skupaj):
    # Filtriraj filme, ki so prejeli oskarja za najboljši mednarodni celovečernji film
    filmi_z_oskarjem = skupaj[(skupaj['kategorija'] == 'International Feature Film') & (skupaj['oskar'] == True)]

    # Dobimo seznam unikatnih naslovov
    return filmi_z_oskarjem['naslov_nom'].dropna().unique()


# Filmi, ki so prejeli oskarja za najboljši mednarodni celovečernji film
odgovor2g = oskarji_za_najboljsi_film(tuji_filmi_dat, zdruzeni_podatki)
print(odgovor2g)



[]
